# Предсказания по модели на PySpark

Исходные данные разделим на две части: в кассандру и кафку. При этом добавим столец id, чтобы объединять по нему после. В конце сделаем стрим из кафки, где будем одновременно соединять разделённый датасет и делать предсказания по заранее сохранённой модели машинного обучения.

In [1]:
! hdfs dfs -rm -r /data
! hdfs dfs -mkdir -p /data/diabet
! hdfs dfs -put diabetes.csv /data/diabet/db

Deleted /data


In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,com.datastax.spark:spark-cassandra-connector_2.12:3.2.0 pyspark-shell'

import findspark
findspark.init()

import time
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StringType, IntegerType, DoubleType, BooleanType
from pyspark.sql.functions import monotonically_increasing_id
from cassandra.cluster import Cluster # https://github.com/datastax/python-driver
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import col as Fcol
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, CountVectorizer, StringIndexer, IndexToString
from pyspark.ml.functions import vector_to_array

from tools_pyspark_hdfs import Spark_HDFS as HDFS
from tools_kafka import Kafka
from tools_pyspark import stop_all_streams, sink, read_stream_kafka, console_stream, console_clear, console_show
from tools_cassandra import schema_cassandra_table

# В качестве датасета выбран: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

spark = SparkSession.builder.appName("my_spark").getOrCreate()
hdfs = HDFS(spark)
kf = Kafka()
topic_name = 'lesson8'

# колонка Pregnancies принудительно делалась категориальной
schema = StructType() \
    .add("Pregnancies", StringType()) \
    .add("Glucose", IntegerType()) \
    .add("BloodPressure", IntegerType()) \
    .add("SkinThickness", IntegerType()) \
    .add("Insulin", IntegerType()) \
    .add("BMI", DoubleType()) \
    .add("DiabetesPedigreeFunction", DoubleType()) \
    .add("Age", IntegerType()) \
    .add("Outcome", IntegerType())

target_column_name = 'Outcome' # это целевая колонка, которую надо предсказать

df = spark.read.csv(
    "/data/diabet/db",
    header=True,
    schema=schema
)

for col in df.columns:  # часть данных будем писать в касандру, а она понимает только строчные буквы 
    df = df.withColumnRenamed(col, col.lower())

target_column_name = 'Outcome' # это целевая колонка, которую надо предсказать
categorical_columns = [f.name for f in schema.fields if 'StringType' in str(f.typeName)]
number_columns = list(set(schema.fieldNames()) - set(categorical_columns))
number_columns.remove(target_column_name) # убираем целевую колонку

df = df.select("*").withColumn("id", monotonically_increasing_id())

# Разделим фрейм на два по колонкам. При этом в каждом оставим колонку id,
df_kafka = df.select('id', 'pregnancies', 'glucose', 'bloodpressure', 'skinthickness') # это запишем в кафку
df_cassandra = df.select('id', 'insulin', 'bmi', 'diabetespedigreefunction', 'age') # это в кассандру
df_kafka.show(5)
df_cassandra.show(5)

22/12/11 14:23:28 WARN Utils: Your hostname, alex resolves to a loopback address: 127.0.1.1; using 192.168.1.70 instead (on interface wlan0)


Ivy Default Cache set to: /root/.ivy2/cache

+---+-----------+-------+-------------+-------------+
| id|pregnancies|glucose|bloodpressure|skinthickness|
+---+-----------+-------+-------------+-------------+
|  0|          6|    148|           72|           35|
|  1|          1|     85|           66|           29|
|  2|          8|    183|           64|            0|
|  3|          1|     89|           66|           23|
|  4|          0|    137|           40|           35|
+---+-----------+-------+-------------+-------------+
only showing top 5 rows

+---+-------+----+------------------------+---+
| id|insulin| bmi|diabetespedigreefunction|age|
+---+-------+----+------------------------+---+
|  0|      0|33.6|                   0.627| 50|
|  1|      0|26.6|                   0.351| 31|
|  2|      0|23.3|                   0.672| 32|
|  3|     94|28.1|                   0.167| 21|
|  4|    168|43.1|                   2.288| 33|
+---+-------+----+------------------------+---+
only showing top 5 rows



## Загрузим данные в таблицу кассандры

In [3]:
# Создадим нужную таблицу
cluster = Cluster(['localhost'])
session = cluster.connect()

session.execute("DROP KEYSPACE IF EXISTS lesson8;")
session.execute("CREATE KEYSPACE IF NOT EXISTS lesson8 WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor': 1 };")
session.execute("USE lesson8;")
session.execute(f"CREATE TABLE IF NOT EXISTS db ( {schema_cassandra_table(df_cassandra, primary_key='id')} );")

cluster.shutdown()

# запишем фрейм в таблицу
df_cassandra.write \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="db", keyspace="lesson8") \
    .mode("append") \
    .save()

# для контроля прочитаем таблицу
spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="db", keyspace="lesson8") \
    .load() \
    .show(5)

+---+---+---+------------------------+-------+
| id|age|bmi|diabetespedigreefunction|insulin|
+---+---+---+------------------------+-------+
| 52| 30| 24|                       0|     23|
|311| 22| 39|                       0|    148|
|732| 24| 44|                       0|    120|
|472| 22| 38|                       0|      0|
|283| 47| 30|                       0|      0|
+---+---+---+------------------------+-------+
only showing top 5 rows



## Загрузим данные в кафку

In [4]:
# создадим топик кафки
if topic_name not in kf.ls():
    kf.add(topic_name)

# отправим данные в топик кафки
stream = df_kafka \
        .selectExpr("CAST(null AS STRING) as key", 
                    "CAST(to_json(struct(*)) AS STRING) as value") \
        .write \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kf.SERVERS) \
        .option("topic", topic_name) \
        .option("checkpointLocation", "checkpoints/stream_read_write") \
        .save()

# посмотрим содержимое топика кафки
kf.get(topic_name, return_rows=True)[:5]

['{"id":0,"pregnancies":"6","glucose":148,"bloodpressure":72,"skinthickness":35}',
 '{"id":1,"pregnancies":"1","glucose":85,"bloodpressure":66,"skinthickness":29}',
 '{"id":2,"pregnancies":"8","glucose":183,"bloodpressure":64,"skinthickness":0}',
 '{"id":3,"pregnancies":"1","glucose":89,"bloodpressure":66,"skinthickness":23}',
 '{"id":4,"pregnancies":"0","glucose":137,"bloodpressure":40,"skinthickness":35}']

### Сделаем поток из кафки, к которому будут добавляться данные из кассандры

На объединённых данных будет делаться предсказание наличия диабета

In [7]:
# Загружаем из файла готовую модель для предсказаний  из HDFS
pipeline_model = PipelineModel.load("my_LR_model")

# DataFrame для запросов к касандре с историческими данными
df_cassandra = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="db", keyspace="lesson8") \
    .load()

#вся логика в этом foreachBatch
def writer_logic(df, epoch_id):
    df.persist()
    df_cassandra.persist()
    
    joined = df.join(df_cassandra, "id", "left") \
        .select(
            Fcol("pregnancies").alias("Pregnancies"),
            Fcol("glucose").alias("Glucose"),
            Fcol("bloodpressure").alias("BloodPressure"),
            Fcol("skinthickness").alias("SkinThickness"),
            Fcol("insulin").alias("Insulin"),
            Fcol("bmi").alias("BMI"),
            Fcol("diabetespedigreefunction").alias("DiabetesPedigreeFunction"),
            Fcol("age").alias("Age"))

    # колонки на предсказании и обучении должны быть одинаковыми.
    # Поэтому делаем точно такие же преобразования с батчем,
    # что и при обучении модели.
    pred =  pipeline_model.transform(joined).select('Pregnancies', 
                                                    'Glucose', 
                                                    'BloodPressure', 
                                                    'SkinThickness', 
                                                    'Insulin', 
                                                    'BMI', 
                                                    'DiabetesPedigreeFunction', 
                                                    'Age', 
                                                    Fcol('category').alias('Diabet'))
    
    console_stream(pred)  # выводим в файловую консоль
    
    # Можно записать данные обратно в кассандру, чтобы по ним же тренировать модель дальше:
    # predict_short.write \
    #     .format("org.apache.spark.sql.cassandra") \
    #     .options(table="db", keyspace="lesson8") \
    #     .mode("append") \
    #     .save()
    
    df_cassandra.unpersist()
    df.unpersist()

# читаем стрим из кафки
stream_kafka = read_stream_kafka(spark, 
                                 server=kf.SERVERS, 
                                 topic_name=topic_name, 
                                 schema=df_kafka.schema, 
                                 maxOffsetsPerTrigger=1)

#связываем источник Кафки и Кассандры через foreachBatch функцию
stream = stream_kafka \
    .writeStream \
    .trigger(processingTime='1 seconds') \
    .foreachBatch(writer_logic) \
    .option("checkpointLocation", "checkpoint/lesson8")

# Запускаем поток
stream = stream.start()
# ждём
time.sleep(15)
# останавливаем поток
stop_all_streams(spark)
# проверяем содержимое консоли
console_show(spark)
# удаляем консоль и чекпойнты
console_clear(spark)
hdfs.rm('checkpoint')

Stopping stream: <pyspark.sql.streaming.StreamingQuery object at 0x7141e095f010>
+-----------+-------+-------------+-------------+-------+---+------------------------+---+------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Diabet|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+------+
|6          |148    |72           |35           |0      |33 |0                       |50 |0     |
|1          |85     |66           |29           |0      |26 |0                       |31 |0     |
+-----------+-------+-------------+-------------+-------+---+------------------------+---+------+

Файл успешно удалён: console
Файл успешно удалён: checkpoint


True